 Import necessary libraries

In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import gdown
import warnings
warnings.filterwarnings('ignore')   
import requests
from io import StringIO

In [6]:
orig_url = "https://drive.google.com/file/d/18MkcxqmccED9QQYXVSPV6kTrC-cEEQgs/view?usp=share_link"
file_id = orig_url.split('/')[-2]
dwn_url = f"https://drive.google.com/uc?export=download&id={file_id}"

response = requests.get(dwn_url)
data = StringIO(response.text)
df = pd.read_csv(data)

In [7]:
df.head()

,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,5 Series,2016,Asia,Red,Petrol,Manual,3.5,151748,98740,8300,High
1,i8,2013,North America,Red,Hybrid,Automatic,1.6,121671,79219,3428,Low
2,5 Series,2022,North America,Blue,Petrol,Automatic,4.5,10991,113265,6994,Low
3,X3,2024,Middle East,Blue,Petrol,Automatic,1.7,27255,60971,4047,Low
4,7 Series,2020,South America,Black,Diesel,Manual,2.1,122131,49898,3080,Low


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Model                 50000 non-null  object 
 1   Year                  50000 non-null  int64  
 2   Region                50000 non-null  object 
 3   Color                 50000 non-null  object 
 4   Fuel_Type             50000 non-null  object 
 5   Transmission          50000 non-null  object 
 6   Engine_Size_L         50000 non-null  float64
 7   Mileage_KM            50000 non-null  int64  
 8   Price_USD             50000 non-null  int64  
 9   Sales_Volume          50000 non-null  int64  
 10  Sales_Classification  50000 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 4.2+ MB


In [9]:
# mean price for each model has been calculated and replaced in the 'Model' column for simplicity
df['Model'] = df.groupby('Model')['Price_USD'].transform('mean')

In [10]:
# Unnest Fuel_Type column
df['Hybrid'] = np.where(df['Fuel_Type']=='Hybrid', 1, 0)
df['Diesel'] = np.where(df['Fuel_Type']=='Diesel', 1, 0)
df['Petrol'] = np.where(df['Fuel_Type']=='Petrol', 1, 0)
df['Electric'] = np.where(df['Fuel_Type']=='Electric', 1, 0)
df.drop('Fuel_Type', axis=1, inplace=True)  

In [11]:
# Unnest Transmission column
df['Automatic'] = np.where(df['Transmission']=='Automatic', 1, 0)
df['Manual'] = np.where(df['Transmission']=='Manual', 1, 0)
df.drop('Transmission', axis=1, inplace=True)

In [12]:
# Unnest Region column
df['Region_North America'] = np.where(df['Region']=='North America', 1, 0)
df['Region_South America'] = np.where(df['Region']=='South America', 1, 0)
df['Region_Europe'] = np.where(df['Region']=='Europe', 1, 0)
df['Region_Asia'] = np.where(df['Region']=='Asia', 1, 0)
df['Region_Africa'] = np.where(df['Region']=='Africa', 1, 0)
df['Region_Middle East'] = np.where(df['Region']=='Middle East', 1, 0)  
df.drop('Region', axis=1, inplace=True) 

In [13]:
#Unnest Color column
df['Color_Black'] = np.where(df['Color']=='Black', 1, 0)
df['Color_White'] = np.where(df['Color']=='White', 1, 0)
df['Color_Blue'] = np.where(df['Color']=='Blue', 1, 0)
df['Color_Red'] = np.where(df['Color']=='Red', 1, 0)
df['Color_Silver'] = np.where(df['Color']=='Silver', 1, 0)
df['Color_Grey'] = np.where(df['Color']=='Grey', 1, 0)
df.drop('Color', axis=1, inplace=True)

In [14]:
# OHE for Sales_Classification
df['Sales_Classification'] = df['Sales_Classification'].map({'Low':0, 'Medium':1, 'High':2})

In [15]:
df.head()

,Model,Year,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification,Hybrid,Diesel,Petrol,...,Region_Europe,Region_Asia,Region_Africa,Region_Middle East,Color_Black,Color_White,Color_Blue,Color_Red,Color_Silver,Color_Grey
0,75287.844077,2016,3.5,151748,98740,8300,2,0,0,1,...,0,1,0,0,0,0,0,1,0,0
1,75366.270951,2013,1.6,121671,79219,3428,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,75287.844077,2022,4.5,10991,113265,6994,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,75016.616856,2024,1.7,27255,60971,4047,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0
4,75570.196742,2020,2.1,122131,49898,3080,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [16]:
df['Sales_Classification'].value_counts()

Sales_Classification
0    34754
2    15246
Name: count, dtype: int64

In [17]:
# Reaarange columns
cols = ['Model', 'Year', 'Mileage_KM', 'Engine_Size_L', 'Hybrid', 'Diesel', 'Petrol', 'Electric', 'Automatic', 'Manual', 'Color_Black', 'Color_White', 'Color_Blue', 'Color_Red', 'Color_Silver', 'Color_Grey', 'Region_North America', 'Region_South America', 'Region_Europe', 'Region_Asia', 'Region_Africa', 'Region_Middle East', 'Price_USD', 'Sales_Classification']
df = df[cols]

In [18]:
df.head()

,Model,Year,Mileage_KM,Engine_Size_L,Hybrid,Diesel,Petrol,Electric,Automatic,Manual,...,Color_Silver,Color_Grey,Region_North America,Region_South America,Region_Europe,Region_Asia,Region_Africa,Region_Middle East,Price_USD,Sales_Classification
0,75287.844077,2016,151748,3.5,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,98740,2
1,75366.270951,2013,121671,1.6,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,79219,0
2,75287.844077,2022,10991,4.5,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,113265,0
3,75016.616856,2024,27255,1.7,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,60971,0
4,75570.196742,2020,122131,2.1,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,49898,0


In [19]:
# Min-Max Scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [20]:
df

,Model,Year,Mileage_KM,Engine_Size_L,Hybrid,Diesel,Petrol,Electric,Automatic,Manual,...,Color_Silver,Color_Grey,Region_North America,Region_South America,Region_Europe,Region_Asia,Region_Africa,Region_Middle East,Price_USD,Sales_Classification
0,0.751362,0.428571,0.758752,0.571429,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.763795,1.0
1,0.820424,0.214286,0.608361,0.028571,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.546890,0.0
2,0.751362,0.857143,0.054942,0.857143,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.925187,0.0
3,0.512521,1.000000,0.136265,0.057143,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.344130,0.0
4,1.000000,0.714286,0.610661,0.171429,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.221094,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.322005,0.285714,0.755161,0.885714,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.143692,1.0
49996,0.322005,0.928571,0.736991,0.771429,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.207938,1.0
49997,0.751362,0.000000,0.874711,0.857143,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.179182,1.0
49998,0.322005,0.714286,0.016881,0.657143,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.317407,1.0


In [21]:
X_classification = df[df.columns.drop(['Sales_Classification', 'Price_USD'])]
y_classification = df['Sales_Classification']  

In [23]:
# Convert to csv
X_classification.to_csv('X_classification.csv', index=False)    
y_classification.to_csv('y_classification.csv', index=False)
# Save preprocessed data

In [22]:
X_Regression = df[df.columns.drop(['Sales_Classification', 'Price_USD'])]
y_Regression = df['Price_USD']

In [24]:
# Convert to csv
X_Regression.to_csv('X_Regression.csv', index=False)    
y_Regression.to_csv('y_Regression.csv', index=False)
# Save preprocessed data    